In [1]:
import numpy as np
import keras.backend as k
from keras.models import Sequential, Model
from keras.layers import Activation
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.applications import vgg16
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
%matplotlib inline

Using TensorFlow backend.


In [2]:
#Set Random Seeds to Constant For Sake of Reproducibility 
import numpy as np
import tensorflow as tf
import random as rn
import os

os.environ['PYTHONHASHSEED']= '0'
np.random.seed(0)
rn.seed(0)
tf.set_random_seed(0)

In [3]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [4]:
train_path = r'Data\TrainData'
valid_path = r'Data\ValidData'
test_path = r'Data\TestData'

In [5]:
train_dataGen = ImageDataGenerator(rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.1,
                     horizontal_flip=True,
                     vertical_flip=True,
                     preprocessing_function=vgg16.preprocess_input
                )
valid_dataGen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)
test_dataGen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)

In [ ]:
train_batches = train_dataGen.flow_from_directory(directory=train_path, target_size=(224, 224), classes=['Benign', 'Malignant'], class_mode='binary', batch_size=10)
valid_batches = valid_dataGen.flow_from_directory(directory=valid_path, target_size=(224, 224), classes=['Benign', 'Malignant'], class_mode='binary', batch_size=10)
test_batches = test_dataGen.flow_from_directory(directory=test_path, target_size=(224, 224), classes=['Benign', 'Malignant'], class_mode='binary', batch_size=10, shuffle=False)

In [ ]:
model = Sequential()
model.add(Conv2D(16, (5, 5), input_shap=(None, None, None, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activaiton='sigmoid'))

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), 'binary_crossentropy', metrics=['accuracy'])
model.fit_generator(generator=train_batches, steps_per_epoch=240, epochs=5, validation_data=valid_batches, validation_steps=24)